In [1]:
import sqlite3
import pandas as pd

# Chargement CSV
ARTICLES_FILE = "arxiv_cs_2020_2025_articles_clean.csv"
AUTHORS_FILE = "arxiv_cs_2020_2025_authors_clean_expanded.csv"

df_articles = pd.read_csv(ARTICLES_FILE, encoding='utf-8-sig')
df_authors = pd.read_csv(AUTHORS_FILE, encoding='utf-8-sig')

# Connexion SQLite (fichier local)
conn = sqlite3.connect("arxiv_relational.db")
cursor = conn.cursor()

# Supprimer tables si elles existent
cursor.executescript("""
DROP TABLE IF EXISTS articles;
DROP TABLE IF EXISTS authors;
DROP TABLE IF EXISTS article_author;
""")

# Création tables
cursor.execute("""
CREATE TABLE articles (
    article_id TEXT PRIMARY KEY,
    submitter TEXT,
    authors TEXT,
    title TEXT,
    journal_ref TEXT,
    doi TEXT,
    report_no TEXT,
    categories TEXT,
    license TEXT,
    abstract TEXT,
    update_date TEXT,
    year INTEGER
)
""")

cursor.execute("""
CREATE TABLE authors (
    author_id INTEGER PRIMARY KEY AUTOINCREMENT,
    author_name TEXT UNIQUE
)
""")

cursor.execute("""
CREATE TABLE article_author (
    article_id TEXT,
    author_id INTEGER,
    PRIMARY KEY(article_id, author_id),
    FOREIGN KEY(article_id) REFERENCES articles(article_id),
    FOREIGN KEY(author_id) REFERENCES authors(author_id)
)
""")

conn.commit()

# Préparation des données articles (remplacer NaN par '')
articles_data = df_articles.fillna('')
articles_data = articles_data.rename(columns={'id':'article_id', 'journal-ref':'journal_ref', 'report-no':'report_no'})

# Insérer articles
articles_to_insert = articles_data[['article_id', 'submitter', 'authors', 'title', 'journal_ref', 'doi', 'report_no',
                                    'categories', 'license', 'abstract', 'update_date', 'year']].values.tolist()

cursor.executemany("""
INSERT INTO articles (article_id, submitter, authors, title, journal_ref, doi, report_no, categories, license, abstract, update_date, year)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", articles_to_insert)

conn.commit()

# Extraire auteurs uniques
authors_unique = df_authors['author_name'].drop_duplicates().reset_index(drop=True).to_frame()

# Insérer auteurs uniques
for author_name in authors_unique['author_name']:
    try:
        cursor.execute("INSERT INTO authors (author_name) VALUES (?)", (author_name,))
    except sqlite3.IntegrityError:
        # ignore duplicates just in case
        pass
conn.commit()

# Charger auteurs avec leur id
authors_db = pd.read_sql("SELECT author_id, author_name FROM authors", conn)

# Construire table article_author avec les ids
df_authors_with_id = pd.merge(df_authors, authors_db, left_on='author_name', right_on='author_name', how='left')
article_author_pairs = df_authors_with_id[['article_id', 'author_id']].drop_duplicates().values.tolist()

cursor.executemany("INSERT OR IGNORE INTO article_author (article_id, author_id) VALUES (?, ?)", article_author_pairs)
conn.commit()

conn.close()

print("✅ Base relationnelle SQLite créée avec succès : 'arxiv_relational.db'")


✅ Base relationnelle SQLite créée avec succès : 'arxiv_relational.db'
